In [7]:
import tifffile
import zarr

In [15]:
z = zarr.open('/mnt/raid_nvme/inference_out/predictions.zarr', mode='r')
print(z.tree())

norms = z['normals_final']
sheet = z['sheet_final']

s_n = norms[:,1000, :, ]
s_s = sheet[1000]

tifffile.imwrite('s_n.tif', s_n)
tifffile.imwrite('s_s.tif', s_s)

/
 ├── normals_count (2000, 1500, 1750) float32
 ├── normals_final (3, 2000, 1500, 1750) uint16
 ├── normals_sum (3, 2000, 1500, 1750) float32
 ├── sheet_count (2000, 1500, 1750) float32
 ├── sheet_final (2000, 1500, 1750) uint8
 └── sheet_sum (2000, 1500, 1750) float32


In [16]:
raw = zarr.open('/mnt/raid_nvme/s1_cutout_outer.zarr', mode='r')
s_r = raw[1000]
tifffile.imwrite('s_r.tif', s_r)

In [14]:
import shutil
import os

def copy_and_number(source_image, start_number, num_copies):
    # Check if source file exists
    if not os.path.exists(source_image):
        print(f"Error: Source file '{source_image}' not found")
        return

    # Get the directory and file extension
    directory = os.path.dirname(source_image) or '.'
    _, ext = os.path.splitext(source_image)

    # Create numbered copies
    for i in range(num_copies):
        new_number = start_number + i
        new_filename = f"{new_number}{ext}"
        new_path = os.path.join(directory, new_filename)

        try:
            shutil.copy2(source_image, new_path)
            print(f"Created: {new_filename}")
        except Exception as e:
            print(f"Error creating {new_filename}: {e}")

# Example usage
if __name__ == "__main__":
    source_image = "/mnt/raid_hdd/s1_segments/5753/20230702185753_inklabels.png"
    start_number = 1
    num_copies =10

    copy_and_number(source_image, start_number, num_copies)

Created: 1.png
Created: 2.png
Created: 3.png
Created: 4.png
Created: 5.png
Created: 6.png
Created: 7.png
Created: 8.png
Created: 9.png
Created: 10.png
